# Generate segmentation
with optimal found parameters

author = Caroline Magg <br>
date = 09 Oktober 2020 <br>

___________________________________
history: <br>
2020-10-09 Test if point data of segmentation can be extracted <br>

In [ ]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cv2
import pydicom
from natsort import natsorted
import scipy
import time
import logging as log
import skimage.segmentation as segmentation
import itertools

In [ ]:
log.basicConfig(format='%(levelname)s:%(message)s', level=log.INFO)

### Add dependencies

In [ ]:
# add KidsBrainProject main folder to paths
sys.path.append(os.path.abspath('../../'))
sys.path.append(os.path.abspath('../utils/'))

In [ ]:
from PatientData import PatientData
from DicomWrapper import DicomWrapper
from Segmentation import Segmentation
from HyperparameterSearchSegmentation import HyperparameterSearchSegmentation as HyperparamSearch

In [ ]:
# add path to data here
path_data = "../../Data/" 

### Contour list

In [ ]:
contours_list = pd.read_csv("list_contours.csv",delimiter=',')
contours_list

### Review the hyperparameter search

In [ ]:
contour_of_interest = contours_list['RoiName'].values
values_of_interest = contours_list['Values'].values
print(contour_of_interest)
print(values_of_interest)
len(contour_of_interest)

# Segmentation T0

In [ ]:
compression_params = [cv2.IMWRITE_PNG_COMPRESSION, 0]

In [ ]:
for idx in range(20,21):
    print("folder", idx)
    t = time.time()
    files = os.listdir(os.path.join(path_data, str(idx)))
    path_contour = os.path.join(path_data, str(idx), 'RS.Jacks{0}.dcm'.format(idx))
    path_preop = os.path.join(path_data, str(idx), 'CT')
    path_postop = [os.path.join(path_data, str(idx), fn) for fn in [x for x in files if 'T1' in x or 'MRT' in x]]
    
    data = PatientData(path_preop, path_postop, path_contour)
    data.read_filtered_contour(roiname=contour_of_interest, mode="exact")
    segmentor = Segmentation(data, debug=True)
    
    directory = os.path.join(path_data, str(idx), 'Segmentation')
    if not os.path.exists(directory):
        os.makedirs(directory)
    if not os.path.exists(os.path.join(directory, 'png')):
        os.makedirs(os.path.join(directory, 'png'))
    if not os.path.exists(os.path.join(directory, 'init')):
        os.makedirs(os.path.join(directory, 'init'))
    if not os.path.exists(os.path.join(directory, 't0')):
        os.makedirs(os.path.join(directory, 't0'))
        
    # dicom image -> png
    print("pngs images")
    img = data.get_pre_images()
    [cv2.imwrite(os.path.join(directory, 'png', '{0}_slice{1}.png'.format(idx,i)), img[i].astype(np.uint8), compression_params) for i in range(len(img))]

    # segmentations
    found_struct = data.get_filtered_contour_names().values
    first_index = data.contour_list_names_filtered['first'].values
    last_index = data.contour_list_names_filtered['last'].values
    # t0 segmentations
    print("t0 segmentation")
    for struct, first, last in zip(found_struct, first_index, last_index):
        tmp = contours_list[contours_list['RoiName'] == struct]['Values'].values[0]
        values = [float(x) for x in tmp.replace('(','').replace(')','').split(',')]
        cont = segmentor.active_contour(struct, postprocess=-1, first=first, last=last,
                       kernel=(int(values[0]),int(values[0])), beta=values[1], max_iterations=int(values[2])) 
        break
    break
        #for i in range(len(cont)):
            #cv2.imwrite(os.path.join(directory, 't0', '{0}_t0_{1}_slice{2}.png'.format(idx,struct,first+i)), 
            #            cont[i].astype(np.uint8), 
            #            compression_params)
        
    # init segmentation
    #print("init segmentation")
    #for ind, first, last in zip(range(len(first_index)), first_index, last_index):
    #    struct = found_struct[ind]
        #for i in range(first, last+1):
            #cv2.imwrite(os.path.join(directory, 'init', '{0}_init_{1}_slice{2}.png'.format(idx,struct,i)),
            #            data.contour_list_names_filtered.iloc[ind]['mask'][i].astype(np.uint8),
            #            compression_params)

In [ ]:
np.shape(segmentor._stack_pts_segm[0])

In [ ]:
np.shape(segmentor._stack_pts_dilated[0])

In [ ]:
np.shape(Segmentation.contour_to_pts(segmentor._stack_contour_init[0])[1])

In [ ]:
plt.imshow(segmentor._stack_contour_init[0])

# Segmentation T1+

# Create csv